## Contents

[Scraping](#scraping)<br>
[MongoDB](#mongo)

Imports

In [1]:
from __future__ import division
from bs4 import BeautifulSoup
import requests
import pymongo

<a id='scraping'></a>
### Scraping

In [26]:
website = 'http://www.snackdata.com'
html = requests.get(website)
soup = BeautifulSoup(html.content, 'html.parser')

In [27]:
#get list of snack urls

#get single div that holds a list
indexholder = soup.findAll(id='indexholder')
#get all "li"s from the div
lis = indexholder[0].findAll('li')
urls = []
for li in lis:
    path = li.find('a')['href']
    urls.append(website + path)

In [28]:
print urls[0]

http://www.snackdata.com/corn_dog


In [29]:
def parse_snack(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    rightstuff = soup.findAll(id='rightstuff')
    name = rightstuff[0].findAll('h1')
    
    snack = dict()
    snack['name'] = name[0].text
    snack['desc'] = rightstuff[0].text
    return snack

<a id='mongo'></a>
### MongoDB

https://docs.mongodb.com/manual/reference/mongo-shell/

In [31]:
mc = pymongo.MongoClient()
db = mc['snacks']
collection = db['snack_data']

In [33]:
for url in urls:
    snack = parse_snack(url)
    collection.insert_one(snack)

In [45]:
print collection.find_one({'name': 'Icing'})

{u'_id': ObjectId('577ddcb690bae047aa9649e9'), u'name': u'Icing', u'desc': u"\nIcing\n\nIcing is a sweet spread that is usually spread upon other sweet foods (like cake or  cupcake). Sometimes it's called frosting, and sometimes it has food coloring added to it. The application of icing is best handled by a professional. If it's applied poorly, it can detract from the presentation of what its trying to enhance (thereby ruining the birthday or special event that it was intended for). \nTasteIcing is sweet because it's made mostly of sugar. It can taste anywhere from terrible to good, depending on the quality of its ingredients. It usually makes a mess, and it's up to the eater to decide whether they like that or not. \n\nIcing became an official snack on April 2nd 2015.\n\n \u2191 Index\n"}


In [39]:
print collection.count()

512


In [48]:
mc.close()